# ML Term Paper - Code: Models
by Ann-Christin and Sarah 15/02/2021

In [2]:
# import libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold
from sklearn.linear_model import Lasso
#pd.set_option("display.max_rows", None, "display.max_columns", None)

In [3]:
import os
path = "C:/Users/gust/Documents/ML_term_paper"  # specify your path here
os.chdir(path)
data = pd.read_csv("Data/piaac_red.csv", sep=',', error_bad_lines=False, index_col=False)
data.shape

C:\Users\gust\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3071: DtypeWarning: Columns (1,2,5,12,18,23,25,26,29,31,32,39,48,51,58,59,60,63,64,66,69,76,85,86,87,93) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


(230691, 130)

In [4]:
data.head()


,Unnamed: 0,cntry,cntry_e,inpiaac,seqid,cnt_brth,ageg10lfs,age_r,gender_r,j_q03b,...,nfe12jr,nfe12njr,fnfaet12,fnfaet12jr,fnfaet12njr,nfehrsnjr,nfehrsjr,nfehrs,nopaidworkever,unempflag
0,0,USA,USA,1.0,9355.0,NaN,NaN,NaN,Male,NaN,...,Did not participate in NFE for JR reasons,Did not participate in NFE for NJR reasons,Did not participate in formal or non-formal AET,Did not participate in formal or non-formal AE...,Did not participate in formal or non-formal AE...,NaN,NaN,NaN,Has had paid work,0.0
1,1,USA,USA,1.0,10882.0,NaN,NaN,NaN,Male,NaN,...,Participated in NFE for JR reasons,Did not participate in NFE for NJR reasons,Participated in formal and/or non-formal AET,Participated in formal or non-formal AET for J...,Did not participate in formal or non-formal AE...,0.0,30.0,30.0,Has had paid work,0.0
2,2,USA,USA,1.0,12329.0,NaN,NaN,NaN,Female,NaN,...,Did not participate in NFE for JR reasons,Did not participate in NFE for NJR reasons,Did not participate in formal or non-formal AET,Did not participate in formal or non-formal AE...,Did not participate in formal or non-formal AE...,NaN,NaN,NaN,Has had paid work,0.0
3,3,USA,USA,1.0,10711.0,NaN,NaN,NaN,Female,NaN,...,Participated in NFE for JR reasons,Did not participate in NFE for NJR reasons,Participated in formal and/or non-formal AET,Participated in formal or non-formal AET for J...,Did not participate in formal or non-formal AE...,NaN,NaN,8.0,Has had paid work,0.0
4,4,USA,USA,1.0,10383.0,NaN,NaN,NaN,Male,NaN,...,Participated in NFE for JR reasons,Did not participate in NFE for NJR reasons,Participated in formal and/or non-formal AET,Participated in formal or non-formal AET for J...,Did not participate in formal or non-formal AE...,0.0,80.0,80.0,Has had paid work,0.0


In [5]:
# data cleaning
dcl = data.drop(columns = ['Unnamed: 0'])

In [7]:
dcl["skilled"] = np.where(dcl["iscoskil4"]==1, 1, 0)
dcl["semi_white"] = np.where(dcl["iscoskil4"]==2, 1, 0)
dcl["semi_blue"] = np.where(dcl["iscoskil4"]==3, 1, 0)
dcl["low_skilled"] = np.where(dcl["iscoskil4"]==4, 1, 0)


In [8]:
dcl = dcl[dcl.iscoskil4 != 'N']
dcl = dcl[dcl.iscoskil4 != 'U']
dcl = dcl[dcl.iscoskil4 != 'A']

In [9]:
dcl.dtypes

cntry           object
cntry_e         object
inpiaac        float64
seqid          float64
cnt_brth        object
                ...   
unempflag      float64
skilled          int32
semi_white       int32
semi_blue        int32
low_skilled      int32
Length: 133, dtype: object

In [34]:
cate = dcl.select_dtypes(include=['object'])
cate = cate.drop(columns = ['iscoskil4'])
cate = list(set(cate))
cate

['gender_r',
 'b_q26a_t',
 'b_q20b',
 'd_q06c',
 'b_d12h',
 'isco1c',
 'aetpop',
 'fnfaet12',
 'b_q01c1_c',
 'd_q09',
 'j_q04a',
 'g_q06',
 'b_q10c',
 'd_q16d5',
 'b_q12g',
 'c_q02a',
 'g_q08',
 'b_q15a',
 'isced_hf_c',
 'fe12',
 'd_q12c',
 'g_q04',
 'c_d09',
 'g_q07',
 'b_q14b',
 'faet12njr',
 'b_q12a',
 'b_q16',
 'b_q01b',
 'b_q01a',
 'computerexperience',
 'd_q06a',
 'd_q04',
 'b_q26b',
 'pared',
 'nopaidworkever',
 'c_q08c1_c',
 'd_q03',
 'd_q05a1_c',
 'b_q02a_t1',
 'd_q05b1_c',
 'c_d06',
 'c_q02c',
 'vet',
 'b_q13',
 'cntry_e',
 'g_q05a',
 'b_q14a',
 'd_q13c',
 'c_d05',
 'b_q11',
 'nfe12njr',
 'b_q02a',
 'leaver1624',
 'g_q05e',
 'nfe12jr',
 'fnfaet12njr',
 'b_q01a_t',
 'd_q07b_c',
 'cnt_brth',
 'b_q05c',
 'isced_hf',
 'd_q04_t',
 'd_q14',
 'faet12jr',
 'f_q07b',
 'isco1l',
 'd_q12a',
 'faet12',
 'b_d01d_c',
 'g_q05f',
 'b_q04a',
 'b_q15c',
 'cnt_h',
 'b_q12e',
 'g_q05d',
 'f_q07a',
 'd_q12b',
 'nfe12',
 'b_q10a',
 'isic1l',
 'ageg10lfs',
 'g_q05h',
 'd_q07a',
 'edcat8',
 'fnfaet1

In [84]:
num = dcl.select_dtypes(include=['float64', "int32"])
num = num.loc[:, ~num.columns.str.startswith('b_q12')]

In [68]:
num.head()
num.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 186619 entries, 1 to 230690
Data columns (total 32 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   inpiaac          2967 non-null    float64
 1   seqid            186619 non-null  float64
 2   age_r            131156 non-null  float64
 3   j_q03b           84970 non-null   float64
 4   j_q03b_c         120495 non-null  float64
 5   racethn_5cat     2960 non-null    float64
 6   yrsqual          176986 non-null  float64
 7   yrsqual_t        172674 non-null  float64
 8   yrsget           122593 non-null  float64
 9   b_q04b           3804 non-null    float64
 10  b_q19a           31026 non-null   float64
 11  c_q08c2          15362 non-null   float64
 12  c_q09            144164 non-null  float64
 13  c_q10a           144106 non-null  float64
 14  d_q05a2          92098 non-null   float64
 15  d_q05b2          17065 non-null   float64
 16  d_q16b           70046 non-null   floa

In [86]:
# drop variables that have too few observations,are not relevant or collinear
num = num.drop(columns = ['inpiaac', "racethn_5cat", "unempflag", "racethn_5cat", "c_q08c2", "seqid", "b_q04b", "b_q19a", 
                         "c_q08c2", "learnatwork", "nfehrsnjr", "nfehrsjr", "nfehrs", "earnmthselfppp", "d_q05b2", 
                          "yrsqual_t", "earnmthbonusppp", "low_skilled"])


In [75]:
num_list = list(set(num))
num_list

['yrsqual',
 'j_q03b',
 'icthome',
 'd_q05a2',
 'age_r',
 'yrsqual_t',
 'ictwork',
 'c_q09',
 'readytolearn',
 'earnmthselfppp',
 'earnmthbonusppp',
 'semi_blue',
 'low_skilled',
 'semi_white',
 'earnmthallppp',
 'yrsget',
 'skilled',
 'j_q03b_c',
 'd_q05b2',
 'd_q16b',
 'c_q10a']

In [54]:
# generate dummies
dummies = [pd.get_dummies(dcl[i], prefix=i, drop_first = True) for i in [cate]]
dummies = dummies.pop(0)
dummies = pd.DataFrame(dummies)
dummies.info()
dummies.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 186619 entries, 1 to 230690
Columns: 672 entries, gender_r_Male to g_q05c_Never
dtypes: uint8(672)
memory usage: 126.0 MB


,gender_r_Male,b_q26a_t_Yes,b_q20b_More than half of the time,b_q20b_None of the time,b_q20b_Up to a quarter of the time,b_q20b_Up to half of the time,d_q06c_Yes,b_d12h_Respondent reported 1 learning activity,b_d12h_Respondent reported learning activities but number is not known,b_d12h_Respondent reported more than 1 learning activity,...,g_q05g_Less than once a month,g_q05g_Less than once a week but at least once a month,g_q05g_Never,b_q02a_t2_Yes,d_q06b_Increased,d_q06b_Stayed more or less the same,g_q05c_Every day,g_q05c_Less than once a month,g_q05c_Less than once a week but at least once a month,g_q05c_Never
1,1,0,0,0,0,0,1,0,0,1,...,0,0,1,0,0,1,1,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,1,0,0,0,0,0,1,0,0,...,0,0,1,0,0,0,1,0,0,0
4,1,1,0,0,0,0,0,0,0,1,...,0,0,1,0,0,0,1,0,0,0
5,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [56]:
# define X and y
on_job = dcl["b_q12d"] #define y here
open_educ = dcl["b_q12f"]
X = num.join(dummies)
X = pd.DataFrame(np.ascontiguousarray(X.values), columns = X.columns)
X.head()

,inpiaac,seqid,age_r,j_q03b,j_q03b_c,racethn_5cat,yrsqual,yrsqual_t,yrsget,b_q04b,...,g_q05g_Less than once a month,g_q05g_Less than once a week but at least once a month,g_q05g_Never,b_q02a_t2_Yes,d_q06b_Increased,d_q06b_Stayed more or less the same,g_q05c_Every day,g_q05c_Less than once a month,g_q05c_Less than once a week but at least once a month,g_q05c_Never
0,1.0,10882.0,NaN,NaN,NaN,2.0,19.0,19.0,19.0,NaN,...,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0
1,1.0,12329.0,NaN,NaN,4.0,3.0,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.0,10711.0,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3,1.0,10383.0,NaN,NaN,1.0,2.0,NaN,NaN,NaN,NaN,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,1.0,12224.0,NaN,NaN,4.0,3.0,9.0,9.0,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [57]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 186619 entries, 0 to 186618
Columns: 704 entries, inpiaac to g_q05c_Never
dtypes: float64(704)
memory usage: 1002.3 MB


In [24]:
X = X.astype(dtype='float64', copy = False)


In [31]:
on_job.describe()

count    45435.000000
mean         3.258105
std          4.441880
min          0.000000
25%          1.000000
50%          2.000000
75%          4.000000
max         50.000000
Name: b_q12d, dtype: float64

In [32]:
open_educ.describe()

count    39488.000000
mean         3.035049
std          3.791765
min          0.000000
25%          1.000000
50%          2.000000
75%          3.000000
max         50.000000
Name: b_q12f, dtype: float64

In [87]:
# run OLS
import statsmodels.api as sm
est = sm.OLS(on_job, num, missing='drop').fit()
X = sm.add_constant(num)
est.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:                 b_q12d   R-squared (uncentered):                   0.409
Model:                            OLS   Adj. R-squared (uncentered):              0.408
Method:                 Least Squares   F-statistic:                              567.9
Date:                Thu, 18 Feb 2021   Prob (F-statistic):                        0.00
Time:                        17:06:35   Log-Likelihood:                         -30144.
No. Observations:               10693   AIC:                                  6.031e+04
Df Residuals:                   10680   BIC:                                  6.041e+04
Df Model:                          13                                                  
Covariance Type:            nonrobust                                                  
=================================================================================
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
age_r            -0.0456      0.009     -5.171      0.000      -0.063      -0.028
j_q03b            0.2162      0.142      1.527      0.127      -0.061       0.494
j_q03b_c         -0.1470      0.161     -0.914      0.361      -0.462       0.168
yrsqual           0.0429      0.022      1.926      0.054      -0.001       0.087
yrsget            0.0870      0.022      4.005      0.000       0.044       0.130
c_q09             0.0342      0.008      4.190      0.000       0.018       0.050
c_q10a            0.1124      0.034      3.285      0.001       0.045       0.179
d_q05a2           0.0007      0.000      4.166      0.000       0.000       0.001
d_q16b        -6.444e-09   5.97e-09     -1.080      0.280   -1.81e-08    5.26e-09
readytolearn      0.3433      0.044      7.792      0.000       0.257       0.430
icthome           0.0028      0.051      0.054      0.957      -0.097       0.103
ictwork           0.0846      0.047      1.809      0.070      -0.007       0.176
earnmthallppp  2.221e-07   2.56e-06      0.087      0.931   -4.79e-06    5.23e-06
skilled                0          0        nan        nan           0           0
semi_white             0          0        nan        nan           0           0
semi_blue              0          0        nan        nan           0           0
==============================================================================
Omnibus:                    12002.845   Durbin-Watson:                   1.950
Prob(Omnibus):                  0.000   Jarque-Bera (JB):          1209326.557
Skew:                           5.809   Prob(JB):                         0.00
Kurtosis:                      53.787   Cond. No.                          inf
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The smallest eigenvalue is      0. This might indicate that there are
strong multicollinearity problems or that the design matrix is singular.
"""

In [ ]:
# feature importance

imp = pd.DataFrame(rf.feature_importances_, columns = ['Feature importance'])
imp['Feature'] = list(X.columns)
imp.sort_values(by=['Feature importance'], ascending=False).reset_index(drop = True)


In [ ]:
# lasso

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1) 
pipe = Pipeline([('scaler', StandardScaler()),
                 ('lasso', 
                  Lasso(alpha=0.005, fit_intercept=True, normalize=False, copy_X=False, max_iter=10000))]) 
# note alpha == lambda in the lecture notes; we just take some value - it is not the optimal! 
# again if you simply follow the defaults, it is not necessary to specify the options
pipe.fit(X_train, y_train)
print('In-sample Rsq: % .4f'
     % pipe.score(X_train, y_train))
print('Out-of-sample Rsq: % .4f'
     % pipe.score(X_test, y_test))
#print(regr.coef_)

In [ ]:
# compare to 